In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tsfresh
!pip install lightgbm
!pip install xgboost

In [ ]:
# Required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
import joblib
warnings.filterwarnings('ignore')

# Advanced data processing
from sklearn.model_selection import (
    train_test_split,
    TimeSeriesSplit,
    RandomizedSearchCV,
    GridSearchCV
)
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, IsolationForest
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping

# Feature engineering
from sklearn.decomposition import PCA
import traceback
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import r2_score
import traceback
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow.keras.layers import BatchNormalization
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [ ]:
class SolarPowerAnalysis:
    def __init__(self):
        self.generation_data = None
        self.weather_data = None
        self.merged_data = None
        self.data_reduction_log = {}

    def log_data_reduction(self, step_name, data, previous_shape=None):
        """
        Log data reduction steps with detailed information
        """
        current_shape = data.shape[0]

        # Handle the reduction calculation
        if previous_shape is None or previous_shape == 0:
            reduction = 0
            reduction_pct = 0
        else:
            reduction = previous_shape - current_shape
            reduction_pct = (reduction / previous_shape) * 100

        self.data_reduction_log[step_name] = {
            'previous_rows': previous_shape,
            'current_rows': current_shape,
            'rows_reduced': reduction,
            'reduction_percentage': reduction_pct
        }

        print(f"\n{step_name}:")
        print(f"Previous rows: {previous_shape if previous_shape else 'N/A'}")
        print(f"Current rows: {current_shape}")
        if previous_shape and previous_shape > 0:
            print(f"Reduction: {reduction} rows ({reduction_pct:.2f}%)")

    def load_data(self, generation_path, weather_path):
        """
        Load and perform initial data processing
        """
        # Load data
        self.generation_data = pd.read_csv(generation_path)
        self.weather_data = pd.read_csv(weather_path)

        # Log initial data sizes
        self.log_data_reduction("Initial Generation Data Load", self.generation_data)
        self.log_data_reduction("Initial Weather Data Load", self.weather_data)

        # Convert timestamps
        for df in [self.generation_data, self.weather_data]:
            if not pd.api.types.is_datetime64_any_dtype(df['DATE_TIME']):
                df['DATE_TIME'] = pd.to_datetime(df['DATE_TIME'])

        # Print sampling frequencies
        gen_freq = self.generation_data['DATE_TIME'].diff().median()
        weather_freq = self.weather_data['DATE_TIME'].diff().median()
        print(f"\nGeneration data frequency: {gen_freq}")
        print(f"Weather data frequency: {weather_freq}")

    def _resample_and_align_data(self):
        """
        Enhanced resampling and alignment with data preservation
        """
        print("\nStarting data resampling and alignment...")

        # 1. Create a complete timestamp range with the higher frequency
        time_range = pd.date_range(
            start=min(self.generation_data['DATE_TIME'].min(),
                      self.weather_data['DATE_TIME'].min()),
            end=max(self.generation_data['DATE_TIME'].max(),
                    self.weather_data['DATE_TIME'].max()),
            freq='15min'
        )

        # 2. Process generation data
        gen_before = self.generation_data.shape[0]

        # First level aggregation with simplified aggregation
        gen_grouped = self.generation_data.groupby(['DATE_TIME', 'SOURCE_KEY']).agg({
            'DC_POWER': ['mean', 'std', 'min', 'max'],
            'AC_POWER': ['mean', 'std', 'min', 'max'],
            'DAILY_YIELD': 'mean',  # Simplified aggregation
            'TOTAL_YIELD': 'mean'   # Simplified aggregation
        }).reset_index()

        # Flatten first level columns for easier access
        gen_grouped.columns = ['DATE_TIME', 'SOURCE_KEY'] + [
            f"{col[0]}_{col[1]}" if isinstance(col, tuple) else col
            for col in gen_grouped.columns[2:]
        ]

        # Plant-level aggregation with simpler aggregation functions
        agg_functions = {
            'DC_POWER_mean': ['sum', 'mean', 'std', 'min', 'max'],
            'AC_POWER_mean': ['sum', 'mean', 'std', 'min', 'max'],
            'DC_POWER_std': 'mean',
            'DC_POWER_min': 'min',
            'DC_POWER_max': 'max',
            'AC_POWER_std': 'mean',
            'AC_POWER_min': 'min',
            'AC_POWER_max': 'max',
            'DAILY_YIELD_mean': 'mean',  # Simplified aggregation
            'TOTAL_YIELD_mean': 'mean'   # Simplified aggregation
        }

        # Aggregate data at the plant level
        gen_agg = gen_grouped.groupby('DATE_TIME').agg(agg_functions).reset_index()

        # Flatten column names after aggregation
        gen_agg.columns = ['DATE_TIME'] + [
            f"{col[0]}_{col[1]}" if isinstance(col, tuple) else col
            for col in gen_agg.columns[1:]
        ]

        # Column name mapping for consistency
        column_mapping = {
            'DC_POWER_mean_sum': 'DC_POWER_sum',
            'AC_POWER_mean_sum': 'AC_POWER_sum',
            'DC_POWER_mean_mean': 'DC_POWER_mean',
            'AC_POWER_mean_mean': 'AC_POWER_mean',
            'DAILY_YIELD_mean_mean': 'DAILY_YIELD',  # Simplified final name
            'TOTAL_YIELD_mean_mean': 'TOTAL_YIELD'   # Simplified final name
        }

        # Rename columns for consistency
        gen_agg = gen_agg.rename(columns=column_mapping)

        # Debugging step: Print column names to check
        print("\nGeneration data columns after aggregation:")
        print(gen_agg.columns.tolist())

        self.log_data_reduction("Generation Data Aggregation", gen_agg, gen_before)

        # 3. Process weather data with multiple interpolation methods
        weather_before = self.weather_data.shape[0]

        # Create different interpolated versions of weather data
        weather_linear = self.weather_data.set_index('DATE_TIME').reindex(time_range)
        weather_linear = weather_linear.interpolate(method='linear', limit_direction='both')

        weather_cubic = self.weather_data.set_index('DATE_TIME').reindex(time_range)
        weather_cubic = weather_cubic.interpolate(method='cubic', limit_direction='both')

        # Combine interpolation methods based on gap size
        weather_resampled = weather_cubic.copy()
        large_gaps = weather_resampled.isna().any(axis=1)
        weather_resampled[large_gaps] = weather_linear[large_gaps]

        # Fill remaining gaps with forward/backward fill
        weather_resampled = weather_resampled.fillna(method='ffill', limit=4)
        weather_resampled = weather_resampled.fillna(method='bfill', limit=4)

        weather_resampled = weather_resampled.reset_index()
        weather_resampled = weather_resampled.rename(columns={'index': 'DATE_TIME'})

        self.log_data_reduction("Weather Data Interpolation", weather_resampled, weather_before)

        # 4. Merge the generation and weather data
        merged_before = gen_agg.shape[0]
        merged = pd.merge(gen_agg, weather_resampled, on='DATE_TIME', how='inner')

        # 5. Calculate efficiency metrics
        merged['DC_AC_RATIO'] = merged['DC_POWER_sum'] / merged['AC_POWER_sum'].replace(0, np.nan)
        merged['PERFORMANCE_RATIO'] = merged['DC_POWER_sum'] / merged['IRRADIATION'].replace(0, np.nan)

        # 6. Handle missing values
        merged = merged.replace([np.inf, -np.inf], np.nan)

        # Interpolate missing numeric values first
        numeric_columns = merged.select_dtypes(include=[np.number]).columns
        merged[numeric_columns] = merged[numeric_columns].interpolate(method='linear', limit_direction='both', limit=4)

        # Fill remaining gaps with forward/backward fill
        merged = merged.fillna(method='ffill', limit=4)
        merged = merged.fillna(method='bfill', limit=4)

        # Drop rows where critical values are missing
        critical_columns = ['DC_POWER_sum', 'AC_POWER_sum', 'IRRADIATION']
        merged = merged.dropna(subset=critical_columns)

        self.log_data_reduction("Final Merge and Cleaning", merged, merged_before)

        return merged

    def engineer_features(self):
        """
        Comprehensive feature engineering
        """
        print("Starting feature engineering...")

        # Resample and merge data
        self.merged_data = self._resample_and_align_data()

        # Check data before feature engineering
        print(f"\nShape before feature engineering: {self.merged_data.shape}")

        # Time-based features
        self.merged_data['hour'] = self.merged_data['DATE_TIME'].dt.hour
        self.merged_data['day'] = self.merged_data['DATE_TIME'].dt.day
        self.merged_data['month'] = self.merged_data['DATE_TIME'].dt.month
        self.merged_data['day_of_week'] = self.merged_data['DATE_TIME'].dt.dayofweek
        self.merged_data['is_weekend'] = self.merged_data['day_of_week'].isin([5, 6]).astype(int)

        # Solar position features
        self.merged_data['day_of_year'] = self.merged_data['DATE_TIME'].dt.dayofyear
        self.merged_data['solar_hour'] = np.sin(2 * np.pi * self.merged_data['hour'] / 24)
        self.merged_data['solar_day'] = np.sin(2 * np.pi * self.merged_data['day_of_year'] / 365)

        # Temperature features
        self.merged_data['temp_diff'] = (self.merged_data['MODULE_TEMPERATURE'] -
                                      self.merged_data['AMBIENT_TEMPERATURE'])

        # Create backup of the data before lag features
        data_before_lag = self.merged_data.copy()

        # Lag features (multiple time steps)
        for lag in [1, 2, 3, 12]:
            self.merged_data[f'DC_POWER_sum_LAG_{lag}'] = self.merged_data['DC_POWER_sum'].shift(lag)
            self.merged_data[f'AMBIENT_TEMPERATURE_LAG_{lag}'] = self.merged_data['AMBIENT_TEMPERATURE'].shift(lag)
            self.merged_data[f'IRRADIATION_LAG_{lag}'] = self.merged_data['IRRADIATION'].shift(lag)

        # Rolling statistics
        for window in [4, 48]:
            self.merged_data[f'DC_POWER_sum_ROLLING_MEAN_{window}'] = (
                self.merged_data['DC_POWER_sum'].rolling(window=window, min_periods=1).mean())
            self.merged_data[f'DC_POWER_sum_ROLLING_STD_{window}'] = (
                self.merged_data['DC_POWER_sum'].rolling(window=window, min_periods=1).std())

        # Handle missing values
        self.merged_data = self.merged_data.replace([np.inf, -np.inf], np.nan)

        # Fill missing values with interpolation first
        self.merged_data = self.merged_data.interpolate(method='linear', limit_direction='both', limit=4)

        # Then use forward/backward fill for any remaining gaps
        self.merged_data = self.merged_data.fillna(method='ffill')
        self.merged_data = self.merged_data.fillna(method='bfill')

        # Check if we lost all data
        if len(self.merged_data) == 0:
            print("Warning: All data was lost during feature engineering!")
            print("Restoring from backup and using simpler feature engineering...")
            self.merged_data = data_before_lag

        print("Feature engineering complete.")
        print(f"Final dataset shape: {self.merged_data.shape}")

        # Print column names for verification
        print("\nAvailable features:")
        for col in self.merged_data.columns:
            print(f"- {col}")

        return self.merged_data

    def detect_anomalies(self):
        """
        Detect anomalies in power generation
        """
        # Isolation Forest for anomaly detection
        iso_forest = IsolationForest(contamination=0.1, random_state=42)

        # Features for anomaly detection
        anomaly_features = ['DC_POWER_sum', 'AC_POWER_sum', 'AMBIENT_TEMPERATURE', 'MODULE_TEMPERATURE', 'IRRADIATION', 'DC_AC_RATIO', 'PERFORMANCE_RATIO']

        # Fit and predict
        anomalies = iso_forest.fit_predict(self.merged_data[anomaly_features])
        self.merged_data['is_anomaly'] = (anomalies == -1).astype(int)

        # Calculate daily anomaly percentage
        daily_anomalies = self.merged_data.groupby(
            self.merged_data['DATE_TIME'].dt.dategenerate_eda_plots
        )['is_anomaly'].mean() * 100

        return daily_anomalies

    def generate_eda_plots(self):
        """ Generate comprehensive EDA plots with advanced analytics """
        plots = {}

        # 1. Time Series Overview
        fig, axes = plt.subplots(3, 1, figsize=(20, 15))
        fig.suptitle('Power Generation Time Series Analysis', fontsize=16)

        # Daily average
        daily_power = self.merged_data.groupby(
            self.merged_data['DATE_TIME'].dt.date)['DC_POWER_sum'].mean()
        daily_power.plot(ax=axes[0], title='Daily Average DC Power')
        axes[0].set_xlabel('Date')
        axes[0].set_ylabel('DC Power (kW)')

        # Weekly average with std
        weekly_power = self.merged_data.groupby(
            pd.Grouper(key='DATE_TIME', freq='W'))['DC_POWER_sum'].agg(['mean', 'std'])
        weekly_power['mean'].plot(ax=axes[1], label='Mean')
        axes[1].fill_between(weekly_power.index,
                            weekly_power['mean'] - weekly_power['std'],
                            weekly_power['mean'] + weekly_power['std'],
                            alpha=0.3, label='±1 std')
        axes[1].set_title('Weekly Average DC Power with Standard Deviation')
        axes[1].set_xlabel('Week')
        axes[1].set_ylabel('DC Power (kW)')
        axes[1].legend()

        # Hourly pattern
        hourly_avg = self.merged_data.groupby('hour')['DC_POWER_sum'].mean()
        hourly_std = self.merged_data.groupby('hour')['DC_POWER_sum'].std()
        axes[2].plot(hourly_avg.index, hourly_avg.values, 'b-', label='Mean')
        axes[2].fill_between(hourly_avg.index,
                            hourly_avg - hourly_std,
                            hourly_avg + hourly_std,
                            alpha=0.3, label='±1 std')
        axes[2].set_title('Average Daily Power Generation Pattern')
        axes[2].set_xlabel('Hour of Day')
        axes[2].set_ylabel('DC Power (kW)')
        axes[2].legend()

        plt.tight_layout()
        plots['time_series_analysis'] = plt.gcf()
        plt.close()

        # 2. Correlation Analysis
        plt.figure(figsize=(15, 12))

        # Select important features
        important_features = [
            'DC_POWER_sum', 'AC_POWER_sum', 'AMBIENT_TEMPERATURE',
            'MODULE_TEMPERATURE', 'IRRADIATION', 'DC_AC_RATIO',
            'PERFORMANCE_RATIO', 'temp_diff',
            'DC_POWER_sum_ROLLING_MEAN_4', 'DC_POWER_sum_ROLLING_STD_4'
        ]

        # Create correlation matrix
        corr_matrix = self.merged_data[important_features].corr()

        # Create mask for upper triangle
        mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

        # Create heatmap with annotations
        sns.heatmap(corr_matrix, mask=mask, annot=True, cmap='coolwarm',
                    fmt='.2f', square=True, linewidths=0.5)
        plt.title('Feature Correlation Analysis', pad=20)
        plots['correlation_analysis'] = plt.gcf()
        plt.close()

        # 3. Environmental Impact Analysis
        fig, axes = plt.subplots(2, 2, figsize=(20, 15))
        fig.suptitle('Environmental Factors Impact Analysis', fontsize=16)

        # Temperature Impact
        sns.scatterplot(data=self.merged_data,
                      x='AMBIENT_TEMPERATURE',
                      y='DC_POWER_sum',
                      alpha=0.5, ax=axes[0, 0])
        axes[0, 0].set_title('Temperature vs Power Generation')

        # Irradiation Impact
        sns.scatterplot(data=self.merged_data,
                      x='IRRADIATION',
                      y='DC_POWER_sum',
                      alpha=0.5, ax=axes[0, 1])
        axes[0, 1].set_title('Irradiation vs Power Generation')

        # Efficiency Analysis
        sns.boxplot(data=self.merged_data,
                    x='hour',
                    y='PERFORMANCE_RATIO',
                    ax=axes[1, 0])
        axes[1, 0].set_title('Performance Ratio by Hour')

        # Temperature Difference Impact
        sns.scatterplot(data=self.merged_data,
                      x='temp_diff',
                      y='DC_POWER_sum',
                      alpha=0.5, ax=axes[1, 1])
        axes[1, 1].set_title('Temperature Difference vs Power Generation')

        plt.tight_layout()
        plots['environmental_analysis'] = plt.gcf()
        plt.close()

        # 4. Performance Metrics Distribution
        fig, axes = plt.subplots(2, 2, figsize=(20, 15))
        fig.suptitle('Performance Metrics Distribution Analysis', fontsize=16)

        # DC Power Distribution
        sns.histplot(data=self.merged_data, x='DC_POWER_sum',
                    kde=True, ax=axes[0, 0])
        axes[0, 0].set_title('DC Power Distribution')

        # AC Power Distribution
        sns.histplot(data=self.merged_data, x='AC_POWER_sum',
                    kde=True, ax=axes[0, 1])
        axes[0, 1].set_title('AC Power Distribution')

        # Performance Ratio Distribution
        sns.histplot(data=self.merged_data, x='PERFORMANCE_RATIO',
                    kde=True, ax=axes[1, 0])
        axes[1, 0].set_title('Performance Ratio Distribution')

        # DC/AC Ratio Distribution
        sns.histplot(data=self.merged_data, x='DC_AC_RATIO',
                    kde=True, ax=axes[1, 1])
        axes[1, 1].set_title('DC/AC Ratio Distribution')

        plt.tight_layout()
        plots['distribution_analysis'] = plt.gcf()
        plt.close()

        # 5. Rolling Statistics Analysis
        fig, axes = plt.subplots(2, 1, figsize=(20, 12))
        fig.suptitle('Rolling Statistics Analysis', fontsize=16)

        # Plot rolling mean
        self.merged_data.set_index('DATE_TIME')[['DC_POWER_sum',
                                              'DC_POWER_sum_ROLLING_MEAN_48']].plot(ax=axes[0])
        axes[0].set_title('DC Power vs 48-period Rolling Mean')
        axes[0].set_xlabel('Date')
        axes[0].set_ylabel('Power (kW)')

        # Plot rolling standard deviation
        self.merged_data['DC_POWER_sum_ROLLING_STD_48'].plot(ax=axes[1])
        axes[1].set_title('48-period Rolling Standard Deviation')
        axes[1].set_xlabel('Date')
        axes[1].set_ylabel('Standard Deviation')

        plt.tight_layout()
        plots['rolling_statistics'] = plt.gcf()
        plt.close()

        return plots

In [ ]:
class SolarPowerModeling:
    def __init__(self, data_processor):
        self.data_processor = data_processor
        self.models = {}
        self.predictions = {}
        self.feature_importance = {}
        self.cv_results = {}
        self.training_logs = {}

    def log_training_step(self, step_name, details):
        """
        Log training progress and parameters
        """
        self.training_logs[step_name] = details
        print(f"\n{step_name}:")
        for key, value in details.items():
            print(f"{key}: {value}")

    def prepare_data(self, test_size=0.15, validation_size=0.15):
        """
        Enhanced data preparation with better feature selection
        """
        # Verify data exists and is not empty
        if self.data_processor.merged_data is None or len(self.data_processor.merged_data) == 0:
            raise ValueError("No data available for model training")

        print("\nPreparing data for modeling...")
        print(f"Total samples available: {len(self.data_processor.merged_data)}")

        # Print all available columns
        print("\nAll available columns:")
        print(self.data_processor.merged_data.columns.tolist())

        # Identify numeric columns
        numeric_cols = self.data_processor.merged_data.select_dtypes(
            include=['float64', 'int64']).columns.tolist()

        # Updated exclude cols
        exclude_cols = [
            'DAILY_YIELD', 'TOTAL_YIELD',  # Updated names
            'PLANT_ID', 'DATE_TIME', 'SOURCE_KEY'
        ]

        # Print columns being excluded
        print("\nColumns being excluded:")
        print(exclude_cols)

        # Create feature columns list
        self.feature_cols = [col for col in numeric_cols
                            if col not in exclude_cols]

        # Print selected features
        print("\nSelected features:")
        print(self.feature_cols)

        # Verify target column exists
        if 'DC_POWER_sum' not in self.data_processor.merged_data.columns:
            raise KeyError("Target column 'DC_POWER_sum' not found in dataset")

        # Use DC_POWER_sum as target variable
        X = self.data_processor.merged_data[self.feature_cols]
        y = self.data_processor.merged_data['DC_POWER_sum']

        # Split data chronologically
        train_size = 1 - (test_size + validation_size)
        train_end = int(len(X) * train_size)
        val_end = int(len(X) * (train_size + validation_size))

        X_train = X[:train_end]
        y_train = y[:train_end]
        X_val = X[train_end:val_end]
        y_val = y[train_end:val_end]
        X_test = X[val_end:]
        y_test = y[val_end:]

        # Scale features using RobustScaler
        scaler = RobustScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_val_scaled = scaler.transform(X_val)
        X_test_scaled = scaler.transform(X_test)

        # Convert to DataFrames with feature names
        X_train_scaled = pd.DataFrame(X_train_scaled, columns=self.feature_cols)
        X_val_scaled = pd.DataFrame(X_val_scaled, columns=self.feature_cols)
        X_test_scaled = pd.DataFrame(X_test_scaled, columns=self.feature_cols)

        # Store the data
        self.train_data = (X_train_scaled, y_train)
        self.val_data = (X_val_scaled, y_val)
        self.test_data = (X_test_scaled, y_test)
        self.scaler = scaler

        # Log data preparation details
        print("\nData Preparation Details:")
        print(f"Training set size: {X_train_scaled.shape}")
        print(f"Validation set size: {X_val_scaled.shape}")
        print(f"Test set size: {X_test_scaled.shape}")
        print(f"\nNumber of features: {len(self.feature_cols)}")
        print("\nSelected features:")
        for col in self.feature_cols:
            print(f"- {col}")

        # Verify split sizes
        print("\nVerifying data splits:")
        print(f"Training samples: {len(X_train)} ({len(X_train)/len(X)*100:.1f}%)")
        print(f"Validation samples: {len(X_val)} ({len(X_val)/len(X)*100:.1f}%)")
        print(f"Test samples: {len(X_test)} ({len(X_test)/len(X)*100:.1f}%)")

        if len(X_train) == 0 or len(X_val) == 0 or len(X_test) == 0:
            raise ValueError("One or more data splits are empty")

        return X_train_scaled, X_val_scaled, X_test_scaled, y_train, y_val, y_test

    def train_random_forest(self):
        """
        Enhanced Random Forest training with more comprehensive hyperparameter tuning
        """
        print("\nTraining Random Forest model...")

        # Define enhanced parameter grid
        param_grid = {
            'n_estimators': [100, 200, 300],
            'max_depth': [10, 15, 20, None],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
            'max_features': ['sqrt', 'log2', None]
        }

        # Initialize base model
        rf = RandomForestRegressor(
            random_state=42,
            n_jobs=-1,  # Use all available cores
            oob_score=True  # Enable out-of-bag score estimation
        )

        # Configure TimeSeriesSplit with appropriate parameters
        n_splits = 3  # Reduced from 5 to 3
        test_size = int(len(self.train_data[0]) * 0.2)
        tscv = TimeSeriesSplit(
            n_splits=n_splits,
            test_size=test_size,
            gap=0
        )

        # Print split information
        print(f"\nTime Series Split Configuration:")
        print(f"Number of splits: {n_splits}")
        print(f"Test size: {test_size}")
        print(f"Training data size: {len(self.train_data[0])}")

        # Perform randomized search
        random_search = RandomizedSearchCV(
            estimator=rf,
            param_distributions=param_grid,
            n_iter=20,  # Number of parameter settings sampled
            cv=tscv,
            scoring=['neg_mean_squared_error', 'r2'],
            refit='neg_mean_squared_error',
            n_jobs=-1,
            verbose=1,
            random_state=42,
            return_train_score=True
        )

        # Fit the model
        X_train, y_train = self.train_data
        random_search.fit(X_train, y_train)

        # Store results
        self.models['rf'] = random_search.best_estimator_
        self.cv_results['rf'] = random_search.cv_results_

        # Calculate and store feature importance
        feature_importance = pd.DataFrame({
            'feature': self.feature_cols,
            'importance': random_search.best_estimator_.feature_importances_
        }).sort_values('importance', ascending=False)

        self.feature_importance['rf'] = feature_importance

        # Log training results
        self.log_training_step("Random Forest Training", {
            'Best parameters': random_search.best_params_,
            'Best CV score': -random_search.best_score_,  # Convert back from negative MSE
            'OOB score': random_search.best_estimator_.oob_score_,
            'Top 5 features': feature_importance.head().to_dict(),
            'Training duration': 'Completed'
        })

    def train_xgboost(self):
        """
        Enhanced XGBoost training with comprehensive hyperparameter tuning
        """
        print("\nTraining XGBoost model...")

        # Define enhanced parameter grid
        param_grid = {
            'n_estimators': [100, 200, 300],
            'max_depth': [4, 6, 8, 10],
            'learning_rate': [0.01, 0.05, 0.1],
            'subsample': [0.8, 0.9, 1.0],
            'colsample_bytree': [0.8, 0.9, 1.0],
            'min_child_weight': [1, 3, 5],
            'gamma': [0, 0.1, 0.2]
        }

        # Initialize base model
        xgb = XGBRegressor(
            random_state=42,
            n_jobs=-1,
            early_stopping_rounds=20
        )

        # Configure TimeSeriesSplit with appropriate parameters
        n_splits = 3  # Reduced from 5 to 3 to match Random Forest
        test_size = int(len(self.train_data[0]) * 0.2)
        tscv = TimeSeriesSplit(
            n_splits=n_splits,
            test_size=test_size,
            gap=0
        )

        # Print split information
        print(f"\nTime Series Split Configuration:")
        print(f"Number of splits: {n_splits}")
        print(f"Test size: {test_size}")
        print(f"Training data size: {len(self.train_data[0])}")

        # Perform randomized search
        random_search = RandomizedSearchCV(
            estimator=xgb,
            param_distributions=param_grid,
            n_iter=20,
            cv=tscv,
            scoring=['neg_mean_squared_error', 'r2'],
            refit='neg_mean_squared_error',
            n_jobs=-1,
            verbose=1,
            random_state=42,
            return_train_score=True
        )

        # Fit the model with validation data
        X_train, y_train = self.train_data
        X_val, y_val = self.val_data

        random_search.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            verbose=False
        )

        # Store results
        self.models['xgb'] = random_search.best_estimator_
        self.cv_results['xgb'] = random_search.cv_results_

        # Calculate and store feature importance
        feature_importance = pd.DataFrame({
            'feature': self.feature_cols,
            'importance': random_search.best_estimator_.feature_importances_
        }).sort_values('importance', ascending=False)

        self.feature_importance['xgb'] = feature_importance

        # Log training results
        self.log_training_step("XGBoost Training", {
            'Best parameters': random_search.best_params_,
            'Best CV score': -random_search.best_score_,
            'Top 5 features': feature_importance.head().to_dict(),
            'Training duration': 'Completed'
        })

    def prepare_sequence_data(self, X, y, seq_length=6):
        """
        Enhanced sequence data preparation with validation
        """
        if len(X) < seq_length + 1:
            raise ValueError(f"Not enough data points. Have {len(X)}, need at least {seq_length + 1}")

        # Convert DataFrame to numpy array if necessary
        if isinstance(X, pd.DataFrame):
            X = X.values
        if isinstance(y, pd.Series):
            y = y.values

        X_seq, y_seq = [], []

        # Create sequences with overlap
        for i in range(len(X) - seq_length):
            X_seq.append(X[i:(i + seq_length)])
            y_seq.append(y[i + seq_length])

        return np.array(X_seq), np.array(y_seq)

    def train_lstm(self):
        """
        Enhanced LSTM training with improved architecture and training process
        """
        try:
            print("\nTraining LSTM model...")

            # Get training and validation data
            X_train, y_train = self.train_data
            X_val, y_val = self.val_data

            # Initialize scalers
            feature_scaler = RobustScaler()  # Changed to RobustScaler for better handling of outliers
            target_scaler = RobustScaler()

            # Handle NaN and negative values with more careful preprocessing
            X_train = np.clip(X_train, 0, None)
            X_val = np.clip(X_val, 0, None)
            y_train = np.clip(y_train, 0, None)
            y_val = np.clip(y_val, 0, None)

            # Handle outliers before scaling
            def remove_outliers(data, threshold=3):
                z_scores = np.abs((data - np.mean(data)) / np.std(data))
                return np.where(z_scores > threshold, np.mean(data), data)

            X_train = np.apply_along_axis(remove_outliers, 0, X_train)
            X_val = np.apply_along_axis(remove_outliers, 0, X_val)

            # Scale features
            X_train_scaled = feature_scaler.fit_transform(X_train)
            X_val_scaled = feature_scaler.transform(X_val)

            # Scale target
            y_train_scaled = target_scaler.fit_transform(y_train.values.reshape(-1, 1)).flatten()
            y_val_scaled = target_scaler.transform(y_val.values.reshape(-1, 1)).flatten()

            # Create sequences with optimal length
            seq_length = 16  # Increased sequence length
            stride = 1  # Reduced stride for more training samples

            X_train_seq, y_train_seq = [], []
            X_val_seq, y_val_seq = [], []

            # Prepare sequences
            for i in range(0, len(X_train_scaled) - seq_length, stride):
                X_train_seq.append(X_train_scaled[i:i + seq_length])
                y_train_seq.append(y_train_scaled[i + seq_length])

            for i in range(0, len(X_val_scaled) - seq_length, stride):
                X_val_seq.append(X_val_scaled[i:i + seq_length])
                y_val_seq.append(y_val_scaled[i + seq_length])

            X_train_seq = np.array(X_train_seq)
            y_train_seq = np.array(y_train_seq)
            X_val_seq = np.array(X_val_seq)
            y_val_seq = np.array(y_val_seq)

            print(f"Training sequence shape: {X_train_seq.shape}")
            print(f"Validation sequence shape: {X_val_seq.shape}")

            # Build enhanced LSTM model
            model = Sequential([
                # First LSTM layer with increased capacity
                LSTM(128, input_shape=(seq_length, X_train.shape[1]),
                    return_sequences=True,
                    kernel_regularizer=tf.keras.regularizers.l2(0.001)),
                BatchNormalization(),
                Dropout(0.3),

                # Second LSTM layer
                LSTM(64, return_sequences=True,
                    kernel_regularizer=tf.keras.regularizers.l2(0.001)),
                BatchNormalization(),
                Dropout(0.3),

                # Third LSTM layer
                LSTM(32, return_sequences=False,
                    kernel_regularizer=tf.keras.regularizers.l2(0.001)),
                BatchNormalization(),
                Dropout(0.3),

                # Dense layers with skip connections
                Dense(32, activation='relu'),
                BatchNormalization(),
                Dense(16, activation='relu'),
                Dropout(0.2),

                # Output layer
                Dense(1, activation='relu')
            ])

            # Custom loss function combining Huber and MSE
            def custom_loss(y_true, y_pred):
                huber = tf.keras.losses.Huber(delta=1.0)(y_true, y_pred)
                mse = tf.keras.losses.MeanSquaredError()(y_true, y_pred)
                return 0.7 * huber + 0.3 * mse

            # Initialize optimizer with gradient clipping
            optimizer = Adam(
                learning_rate=0.0005,  # Lower initial learning rate
                clipnorm=1.0,
                beta_1=0.9,
                beta_2=0.999,
                epsilon=1e-07
            )

            model.compile(
                optimizer=optimizer,
                loss=custom_loss,
                metrics=['mae', 'mse']
            )

            # Print model summary
            model.summary()

            # Enhanced callbacks
            callbacks = [
                EarlyStopping(
                    monitor='val_loss',
                    patience=15,  # Increased patience
                    restore_best_weights=True,
                    min_delta=0.0001
                ),
                ReduceLROnPlateau(
                    monitor='val_loss',
                    factor=0.2,  # More aggressive reduction
                    patience=7,
                    min_lr=0.00001,
                    verbose=1
                ),
                ModelCheckpoint(
                    'best_lstm_model.keras',
                    monitor='val_loss',
                    save_best_only=True,
                    verbose=1
                )
            ]

            # Train with modified parameters
            history = model.fit(
                X_train_seq, y_train_seq,
                validation_data=(X_val_seq, y_val_seq),
                epochs=50,  # Increased epochs
                batch_size=64,  # Increased batch size
                callbacks=callbacks,
                shuffle=False,
                verbose=1
            )

            # Store model and scalers
            self.models['lstm'] = model
            self.scalers = {
                'feature_scaler': feature_scaler,
                'target_scaler': target_scaler,
                'seq_length': seq_length
            }

            return model, history

        except Exception as e:
            print(f"Error in LSTM training: {str(e)}")
            traceback.print_exc()
            return None, None

    def create_ensemble(self):
        """
        Fixed ensemble creation with proper error handling
        """
        print("\nCreating ensemble model...")

        try:
            # Get available models
            available_models = {name: model for name, model in self.models.items()
                              if model is not None}

            if len(available_models) < 2:
                print("Not enough models available for ensemble creation")
                return None

            # Calculate weights based on validation performance
            weights = {}
            X_val, y_val = self.val_data

            for name, model in available_models.items():
                try:
                    if name == 'lstm':
                        # Handle LSTM predictions separately
                        seq_length = self.scalers['seq_length']
                        X_val_scaled = self.scalers['feature_scaler'].transform(X_val)
                        X_val_seq = np.array([X_val_scaled[i:i + seq_length]
                                            for i in range(len(X_val_scaled) - seq_length)])
                        val_pred = model.predict(X_val_seq).flatten()
                        val_score = r2_score(y_val[seq_length:], val_pred)
                    else:
                        # Handle RF and XGB predictions
                        val_pred = model.predict(X_val)
                        val_score = r2_score(y_val, val_pred)

                    weights[name] = max(val_score, 0)  # Ensure non-negative weights

                except Exception as e:
                    print(f"Error calculating weight for {name}: {str(e)}")
                    continue

            # Normalize weights
            if weights:
                total = sum(weights.values())
                if total > 0:
                    self.ensemble_weights = {k: v/total for k, v in weights.items()}
                    print("Ensemble Creation:")
                    print(f"Model weights: {self.ensemble_weights}")
                    print(f"Number of models: {len(self.ensemble_weights)}")
                    print(f"Included models: {list(self.ensemble_weights.keys())}")
                else:
                    print("All model weights were zero")
                    self.ensemble_weights = None
            else:
                print("No valid models for ensemble creation")
                self.ensemble_weights = None

        except Exception as e:
            print(f"Error in ensemble creation: {str(e)}")
            traceback.print_exc()
            self.ensemble_weights = None

In [ ]:
class ModelEvaluation:
    def __init__(self, model_trainer):
        self.model_trainer = model_trainer
        self.evaluation_results = {}
        self.visualization_results = {}
        self.error_analysis = {}
        self.detailed_metrics = {}
        # Add this line to get scalers from model trainer
        self.scalers = getattr(self.model_trainer, 'scalers', None)

    def calculate_metrics(self, y_true, y_pred, prefix=''):
        """
        Comprehensive metric calculation
        """
        metrics = {
            f'{prefix}RMSE': np.sqrt(mean_squared_error(y_true, y_pred)),
            f'{prefix}MAE': mean_absolute_error(y_true, y_pred),
            f'{prefix}R2': r2_score(y_true, y_pred),
            f'{prefix}MAPE': np.mean(np.abs((y_true - y_pred) / y_true)) * 100,
            f'{prefix}Normalized_RMSE': np.sqrt(mean_squared_error(y_true, y_pred)) / np.mean(y_true),
            f'{prefix}Bias': np.mean(y_pred - y_true),
            f'{prefix}Max_Error': np.max(np.abs(y_pred - y_true))
        }

        # Add time-based metrics
        if len(y_true) > 24:  # If we have at least a day's worth of data
            # Daily metrics
            daily_true = pd.Series(y_true).rolling(window=24).mean()
            daily_pred = pd.Series(y_pred).rolling(window=24).mean()
            metrics[f'{prefix}Daily_RMSE'] = np.sqrt(mean_squared_error(
                daily_true.dropna(), daily_pred.dropna()))

        return metrics

    def _calculate_metrics(self, y_true, y_pred, prefix=''):
        """
        Enhanced metric calculation with better validation
        """
        try:
            # Ensure arrays are numpy arrays
            y_true = np.asarray(y_true)
            y_pred = np.asarray(y_pred)

            # Remove any NaN values
            mask = ~(np.isnan(y_true) | np.isnan(y_pred) |
                    np.isinf(y_true) | np.isinf(y_pred))

            y_true = y_true[mask]
            y_pred = y_pred[mask]

            if len(y_true) == 0 or len(y_pred) == 0:
                raise ValueError("No valid data points for metric calculation")

            # Ensure same length
            min_len = min(len(y_true), len(y_pred))
            y_true = y_true[:min_len]
            y_pred = y_pred[:min_len]

            # Calculate basic metrics with error checking
            metrics = {}

            try:
                metrics[f'{prefix}RMSE'] = np.sqrt(mean_squared_error(y_true, y_pred))
            except Exception as e:
                print(f"Error calculating RMSE: {e}")

            try:
                metrics[f'{prefix}MAE'] = mean_absolute_error(y_true, y_pred)
            except Exception as e:
                print(f"Error calculating MAE: {e}")

            try:
                metrics[f'{prefix}R2'] = r2_score(y_true, y_pred)
            except Exception as e:
                print(f"Error calculating R2: {e}")

            # Calculate normalized RMSE
            try:
                mean_true = np.mean(y_true)
                if mean_true != 0:
                    metrics[f'{prefix}Normalized_RMSE'] = metrics[f'{prefix}RMSE'] / mean_true
            except Exception as e:
                print(f"Error calculating Normalized RMSE: {e}")

            # Calculate MAPE for non-zero values
            try:
                nonzero_mask = y_true != 0
                if np.any(nonzero_mask):
                    y_true_nz = y_true[nonzero_mask]
                    y_pred_nz = y_pred[nonzero_mask]
                    metrics[f'{prefix}MAPE'] = np.mean(np.abs((y_true_nz - y_pred_nz) / y_true_nz)) * 100
            except Exception as e:
                print(f"Error calculating MAPE: {e}")

            # Additional metrics
            try:
                metrics[f'{prefix}Bias'] = np.mean(y_pred - y_true)
                metrics[f'{prefix}Max_Error'] = np.max(np.abs(y_pred - y_true))
            except Exception as e:
                print(f"Error calculating additional metrics: {e}")

            return metrics

        except Exception as e:
            print(f"Error in metric calculation: {str(e)}")
            return {f'{prefix}error': str(e)}

    def evaluate_models(self):
        """
        Fixed model evaluation with proper variable scope
        """
        print("\nEvaluating models...")
        self.evaluation_results = {}
        self.detailed_metrics = {}
        test_length = None  # Initialize test_length at the start

        try:
            X_test, y_test = self.model_trainer.test_data
            original_length = len(y_test)
            print(f"Original test set length: {original_length}")

            # Evaluate LSTM first
            if self.model_trainer.models.get('lstm') is not None:
                try:
                    print("Starting LSTM evaluation...")
                    lstm_metrics, lstm_pred, lstm_test = self.evaluate_lstm(X_test, y_test)
                    if lstm_metrics is not None and lstm_pred is not None:
                        test_length = len(lstm_pred)  # Set test_length from LSTM results
                        print(f"LSTM prediction length: {test_length}")

                        # Store LSTM results
                        self.evaluation_results['lstm'] = lstm_metrics
                        self.detailed_metrics['lstm'] = {
                            'predictions': lstm_pred,
                            'actual': lstm_test,
                            'metrics': lstm_metrics
                        }
                        print("LSTM evaluation complete")
                except Exception as e:
                    print(f"Error in LSTM evaluation: {str(e)}")
                    test_length = original_length  # Fall back to original length
            else:
                test_length = original_length

            # Use test_length for other models
            y_test_adjusted = y_test[-test_length:] if test_length else y_test
            X_test_adjusted = X_test[-test_length:] if test_length else X_test

            # Evaluate RF
            if self.model_trainer.models.get('rf') is not None:
                try:
                    print("Evaluating RF...")
                    rf_pred = self.model_trainer.models['rf'].predict(X_test_adjusted)
                    rf_metrics = self._calculate_metrics(y_test_adjusted, rf_pred, 'rf_')
                    self.evaluation_results['rf'] = rf_metrics
                    self.detailed_metrics['rf'] = {
                        'predictions': rf_pred,
                        'actual': y_test_adjusted,
                        'metrics': rf_metrics
                    }
                    print("RF evaluation complete")
                except Exception as e:
                    print(f"Error in RF evaluation: {str(e)}")

            # Evaluate XGBoost
            if self.model_trainer.models.get('xgb') is not None:
                try:
                    print("Evaluating XGBoost...")
                    xgb_pred = self.model_trainer.models['xgb'].predict(X_test_adjusted)
                    xgb_metrics = self._calculate_metrics(y_test_adjusted, xgb_pred, 'xgb_')
                    self.evaluation_results['xgb'] = xgb_metrics
                    self.detailed_metrics['xgb'] = {
                        'predictions': xgb_pred,
                        'actual': y_test_adjusted,
                        'metrics': xgb_metrics
                    }
                    print("XGB evaluation complete")
                except Exception as e:
                    print(f"Error in XGBoost evaluation: {str(e)}")

            # Create and evaluate ensemble
            if len(self.detailed_metrics) >= 2 and self.model_trainer.ensemble_weights:
                try:
                    print("Creating ensemble predictions...")
                    ensemble_pred = np.zeros_like(y_test_adjusted, dtype=float)

                    for model_name, weight in self.model_trainer.ensemble_weights.items():
                        if model_name in self.detailed_metrics:
                            ensemble_pred += weight * self.detailed_metrics[model_name]['predictions']

                    ensemble_metrics = self._calculate_metrics(y_test_adjusted, ensemble_pred, 'ensemble_')
                    self.evaluation_results['ensemble'] = ensemble_metrics
                    self.detailed_metrics['ensemble'] = {
                        'predictions': ensemble_pred,
                        'actual': y_test_adjusted,
                        'metrics': ensemble_metrics
                    }
                    print("Ensemble evaluation complete")
                except Exception as e:
                    print(f"Error in ensemble evaluation: {str(e)}")

            return self.evaluation_results

        except Exception as e:
            print(f"Error in model evaluation: {str(e)}")
            traceback.print_exc()
            return {'error': str(e)}

    def _evaluate_tree_models(self, model_name, X_test, y_test):
        """
        Enhanced evaluation for tree-based models with detailed error analysis
        """
        try:
            model = self.model_trainer.models[model_name]
            if model is None:
                print(f"Skipping {model_name} evaluation - model not trained")
                return

            # Make predictions
            y_pred = model.predict(X_test)
            if len(y_pred) == 0:
                raise ValueError(f"Empty predictions for {model_name}")

            # Calculate comprehensive metrics
            metrics = self._calculate_comprehensive_metrics(y_test, y_pred, model_name)

            # Calculate time-based metrics
            time_metrics = self._calculate_time_based_metrics(y_test, y_pred)
            metrics.update({f"{model_name}_{k}": v for k, v in time_metrics.items()})

            # Feature importance analysis
            if hasattr(model, 'feature_importances_'):
                feature_imp = pd.DataFrame({
                    'feature': self.model_trainer.feature_cols,
                    'importance': model.feature_importances_
                }).sort_values('importance', ascending=False)

                metrics[f'{model_name}_top_features'] = feature_imp.head(10).to_dict()
                self.feature_importance_analysis[model_name] = feature_imp

            # Error analysis
            error_metrics = self._calculate_error_distribution(y_test, y_pred)
            self.error_analysis_results[model_name] = error_metrics
            metrics.update({f"{model_name}_{k}": v for k, v in error_metrics.items()})

            # Store all results
            self.evaluation_results[model_name] = metrics
            self.detailed_metrics[model_name] = {
                'predictions': y_pred,
                'actual': y_test,
                'metrics': metrics
            }

        except Exception as e:
            print(f"Error evaluating {model_name}: {str(e)}")
            self.evaluation_results[model_name] = {
                'error': f'Evaluation failed for {model_name}',
                'error_message': str(e)
            }

    def _calculate_comprehensive_metrics(self, y_true, y_pred, prefix=''):
        """
        Calculate comprehensive evaluation metrics
        """
        metrics = {
            f'{prefix}RMSE': np.sqrt(mean_squared_error(y_true, y_pred)),
            f'{prefix}MAE': mean_absolute_error(y_true, y_pred),
            f'{prefix}R2': r2_score(y_true, y_pred),
            f'{prefix}MAPE': np.mean(np.abs((y_true - y_pred) / y_true)) * 100,
            f'{prefix}Normalized_RMSE': np.sqrt(mean_squared_error(y_true, y_pred)) / np.mean(y_true),
            f'{prefix}Bias': np.mean(y_pred - y_true),
            f'{prefix}Max_Error': np.max(np.abs(y_pred - y_true))
        }

        # Calculate additional metrics
        metrics[f'{prefix}Median_AE'] = np.median(np.abs(y_true - y_pred))
        metrics[f'{prefix}R2_Adjusted'] = 1 - (1 - metrics[f'{prefix}R2']) * (len(y_true) - 1) / (len(y_true) - len(self.model_trainer.feature_cols) - 1)
        metrics[f'{prefix}RMSE_CV'] = np.sqrt(mean_squared_error(y_true, y_pred)) / np.mean(y_true) * 100

        return metrics

    def _calculate_time_based_metrics(self, y_true, y_pred):
        """
        Calculate metrics for different time periods
        """
        metrics = {}

        # Daily aggregation
        daily_true = pd.Series(y_true).rolling(window=96).mean()  # 24 hours * 4 (15-min intervals)
        daily_pred = pd.Series(y_pred).rolling(window=96).mean()

        # Weekly aggregation
        weekly_true = pd.Series(y_true).rolling(window=96*7).mean()
        weekly_pred = pd.Series(y_pred).rolling(window=96*7).mean()

        # Calculate metrics for different time scales
        metrics['Daily_RMSE'] = np.sqrt(mean_squared_error(daily_true.dropna(), daily_pred.dropna()))
        metrics['Weekly_RMSE'] = np.sqrt(mean_squared_error(weekly_true.dropna(), weekly_pred.dropna()))

        return metrics

    def _calculate_error_distribution(self, y_true, y_pred):
        """
        Analyze error distribution and patterns
        """
        errors = y_true - y_pred
        abs_errors = np.abs(errors)

        metrics = {
            'Error_Mean': np.mean(errors),
            'Error_Std': np.std(errors),
            'Error_Skew': pd.Series(errors).skew(),
            'Error_Kurtosis': pd.Series(errors).kurtosis(),
            'Error_Q1': np.percentile(abs_errors, 25),
            'Error_Q3': np.percentile(abs_errors, 75),
            'Error_IQR': np.percentile(abs_errors, 75) - np.percentile(abs_errors, 25)
        }

        return metrics

    def evaluate_lstm(self, X_test, y_test):
        """
        Enhanced LSTM evaluation with proper scaling handling
        """
        try:
            if isinstance(y_test, pd.Series):
                y_test = y_test.to_numpy()

            # Handle NaN values
            X_test = np.nan_to_num(X_test, nan=0.0, posinf=0.0, neginf=0.0)
            y_test = np.nan_to_num(y_test, nan=0.0, posinf=0.0, neginf=0.0)

            # Scale test data
            X_test_scaled = self.scalers['feature_scaler'].transform(X_test)
            seq_length = self.scalers['seq_length']

            # Create sequences
            X_test_seq = []
            y_test_seq = []

            for i in range(len(X_test_scaled) - seq_length):
                X_test_seq.append(X_test_scaled[i:i + seq_length])
                y_test_seq.append(y_test[i + seq_length])

            X_test_seq = np.array(X_test_seq)
            y_test_seq = np.array(y_test_seq)

            print(f"Test sequence shapes - X: {X_test_seq.shape}, y: {y_test_seq.shape}")

            # Make predictions
            y_pred_scaled = self.model_trainer.models['lstm'].predict(X_test_seq)

            # Inverse transform predictions
            y_pred = self.scalers['target_scaler'].inverse_transform(
                y_pred_scaled).flatten()

            print(f"Final shapes - predictions: {y_pred.shape}, actual: {y_test_seq.shape}")
            print(f"Range - predictions: [{y_pred.min():.2f}, {y_pred.max():.2f}], "
                  f"actual: [{y_test_seq.min():.2f}, {y_test_seq.max():.2f}]")

            # Calculate metrics
            metrics = self._calculate_metrics(y_test_seq, y_pred, 'lstm_')
            print("LSTM metrics calculated successfully")

            return metrics, y_pred, y_test_seq

        except Exception as e:
            print(f"Error in LSTM evaluation: {str(e)}")
            traceback.print_exc()
            return None, None, None

    def _evaluate_ensemble(self, X_test, y_test):
        """
        Evaluate ensemble model with weighted predictions
        """
        if self.model_trainer.ensemble_weights is None:
            print("Skipping ensemble evaluation - ensemble not created")
            return

        try:
            # Get predictions from all models
            predictions = {}

            # Tree-based models
            for model_name in ['rf', 'xgb']:
                if self.model_trainer.models[model_name] is not None:
                    predictions[model_name] = self.model_trainer.models[model_name].predict(X_test)

            # LSTM
            if self.model_trainer.models['lstm'] is not None:
                X_test_seq, y_test_seq = self.model_trainer.prepare_sequence_data(X_test, y_test)
                lstm_pred = self.model_trainer.models['lstm'].predict(X_test_seq).flatten()
                predictions['lstm'] = lstm_pred[-len(predictions['rf']):]

            # Calculate weighted ensemble predictions
            ensemble_pred = np.zeros_like(predictions['rf'])
            for model_name, weight in self.model_trainer.ensemble_weights.items():
                if model_name in predictions:
                    ensemble_pred += weight * predictions[model_name]

            # Calculate metrics
            metrics = self.calculate_metrics(y_test[-len(ensemble_pred):],
                                          ensemble_pred, 'ensemble_')

            # Error analysis
            self.error_analysis['ensemble'] = {
                'residuals': y_test[-len(ensemble_pred):] - ensemble_pred,
                'percent_error': ((y_test[-len(ensemble_pred):] - ensemble_pred) /
                                y_test[-len(ensemble_pred):]) * 100,
                'abs_error': np.abs(y_test[-len(ensemble_pred):] - ensemble_pred)
            }

            self.evaluation_results['ensemble'] = metrics

        except Exception as e:
            print(f"Error in ensemble evaluation: {str(e)}")

    def _perform_comparative_analysis(self):
        """
        Perform comparative analysis between models
        """
        # Compare models on common metrics
        common_metrics = ['RMSE', 'MAE', 'R2', 'MAPE']
        model_comparison = {}

        for metric in common_metrics:
            metric_values = {}
            for model_name, metrics in self.evaluation_results.items():
                metric_key = f'{model_name}_{metric}'
                if metric_key in metrics:
                    metric_values[model_name] = metrics[metric_key]
            model_comparison[metric] = metric_values

        self.evaluation_results['model_comparison'] = model_comparison

    def generate_evaluation_plots(self):
        """
        Fixed plot generation with proper variable handling
        """
        plots = {}

        try:
            # Only create plots if we have valid metrics
            if not self.detailed_metrics:
                print("No metrics available for plotting")
                return {}

            # 1. Prediction vs Actual plots
            fig, axes = plt.subplots(2, 2, figsize=(20, 15))
            fig.suptitle('Prediction vs Actual Comparison', fontsize=16)

            for idx, (model_name, metrics) in enumerate(self.detailed_metrics.items()):
                if 'actual' not in metrics or 'predictions' not in metrics:
                    continue

                row = idx // 2
                col = idx % 2

                actual = metrics['actual']
                pred = metrics['predictions']

                # Create scatter plot
                axes[row, col].scatter(actual, pred, alpha=0.5, label='Predictions')

                # Add perfect prediction line
                min_val = min(np.min(actual), np.min(pred))
                max_val = max(np.max(actual), np.max(pred))
                axes[row, col].plot([min_val, max_val], [min_val, max_val],
                                  'r--', label='Perfect Prediction')

                axes[row, col].set_title(f'{model_name.upper()} Predictions')
                axes[row, col].set_xlabel('Actual Values')
                axes[row, col].set_ylabel('Predicted Values')
                axes[row, col].legend()

            plt.tight_layout()
            plots['prediction_comparison'] = plt.gcf()
            plt.close()

            # 2. Time series plot
            fig, ax = plt.subplots(figsize=(20, 10))

            # Get first model's actual values for reference
            first_model = next(iter(self.detailed_metrics.values()))
            if 'actual' in first_model:
                reference_actual = first_model['actual'][:100]  # Plot first 100 points
                ax.plot(reference_actual, 'k--', label='Actual', alpha=0.5)

            # Plot predictions for each model
            for model_name, metrics in self.detailed_metrics.items():
                if 'predictions' in metrics:
                    pred = metrics['predictions'][:100]
                    ax.plot(pred, label=f'{model_name.upper()} Predictions')

            ax.set_title('Time Series Comparison (First 100 points)')
            ax.set_xlabel('Time Step')
            ax.set_ylabel('Power Output')
            ax.legend()
            plt.tight_layout()
            plots['time_series'] = plt.gcf()
            plt.close()

            return plots

        except Exception as e:
            print(f"Error generating evaluation plots: {str(e)}")
            traceback.print_exc()
            return {}


    def generate_evaluation_report(self):
        """
        Generate a comprehensive evaluation report
        """
        report = {
            'summary': {
                'best_model': None,
                'best_score': float('-inf'),
                'model_rankings': {}
            },
            'detailed_metrics': self.evaluation_results,
            'error_analysis': {
                model: {
                    'mean_error': np.mean(data['residuals']),
                    'std_error': np.std(data['residuals']),
                    'max_error': np.max(np.abs(data['residuals'])),
                    'error_95th_percentile': np.percentile(np.abs(data['residuals']), 95)
                }
                for model, data in self.error_analysis.items()
            }
        }

        # Determine best model
        for model_name, metrics in self.evaluation_results.items():
            if model_name != 'model_comparison':
                score = metrics.get(f'{model_name}_R2', float('-inf'))
                report['summary']['model_rankings'][model_name] = score
                if score > report['summary']['best_score']:
                    report['summary']['best_score'] = score
                    report['summary']['best_model'] = model_name

        return report

def main():
    # Initialize data processor
    data_processor = SolarPowerAnalysis()

    # Load and process data
    data_processor.load_data('/content/drive/MyDrive/Data mining/Plant_1_Generation_Data.csv', '/content/drive/MyDrive/Data mining/Plant_1_Weather_Sensor_Data.csv')
    data_processor.engineer_features()

    # Generate EDA plots
    try:
        eda_plots = data_processor.generate_eda_plots()
        print("Generated EDA plots successfully")
    except Exception as e:
        print(f"Error generating EDA plots: {e}")
        eda_plots = {}

    # Initialize model trainer
    model_trainer = SolarPowerModeling(data_processor)

    # Prepare data for modeling
    model_trainer.prepare_data()

    # Train models
    model_trainer.train_random_forest()
    model_trainer.train_xgboost()
    model_trainer.train_lstm()
    model_trainer.create_ensemble()

    # Initialize evaluator
    evaluator = ModelEvaluation(model_trainer)

    # Evaluate models
    evaluation_results = evaluator.evaluate_models()

    # Generate evaluation plots
    try:
        evaluation_plots = evaluator.generate_evaluation_plots()
        print("Generated evaluation plots successfully")
    except Exception as e:
        print(f"Error generating evaluation plots: {e}")
        evaluation_plots = {}

    # Print results
    print("\nModel Evaluation Results:")
    for model_name, metrics in evaluation_results.items():
        if model_name != 'model_comparison' and isinstance(metrics, dict):
            print(f"\n{model_name.upper()} Results:")
            for metric_name, value in metrics.items():
                try:
                    if isinstance(value, dict):
                        print(f"{metric_name}:")
                        for k, v in value.items():
                            if isinstance(v, (int, float)):
                                print(f"  {k}: {v:.4f}")
                            else:
                                print(f"  {k}: {v}")
                    elif isinstance(value, (int, float)):
                        print(f"{metric_name}: {value:.4f}")
                    else:
                        print(f"{metric_name}: {value}")
                except Exception as e:
                    print(f"Error printing metric {metric_name}: {str(e)}")

    # Save models with error handling
    for model_name, model in model_trainer.models.items():
        if model is not None:
            try:
                if model_name != 'lstm':
                    joblib.dump(model, f'{model_name}_model.joblib')
                else:
                    model.save(f'{model_name}_model.h5')
                print(f"Successfully saved {model_name} model")
            except Exception as e:
                print(f"Error saving {model_name} model: {str(e)}")

    # Save plots
    all_plots = {**eda_plots, **evaluation_plots}
    for plot_name, fig in all_plots.items():
        try:
            fig.savefig(f'{plot_name}.png')
            print(f"Successfully saved plot: {plot_name}")
        except Exception as e:
            print(f"Error saving plot {plot_name}: {str(e)}")

    # Save evaluation results
    try:
        pd.DataFrame(evaluation_results).to_csv('model_evaluation_results.csv')
        print("Successfully saved evaluation results")
    except Exception as e:
        print(f"Error saving evaluation results: {str(e)}")

    # Print final results
    print("\nExecution complete. Results and plots have been saved.")

    # Determine best model based on RMSE
    best_model = None
    best_score = float('inf')
    model_rankings = {}

    for model_name, metrics in evaluation_results.items():
        if model_name != 'model_comparison' and isinstance(metrics, dict):
            rmse = metrics.get(f'{model_name}_RMSE')
            if rmse is not None:
                model_rankings[model_name] = rmse
                if rmse < best_score:
                    best_score = rmse
                    best_model = model_name

    print("\nBest Model:", best_model)
    print("\nModel Rankings:")
    for model_name, score in sorted(model_rankings.items(), key=lambda x: x[1]):
        print(f"{model_name}: RMSE = {score:.4f}")

if __name__ == "__main__":
    main()


Initial Generation Data Load:
Previous rows: N/A
Current rows: 68778

Initial Weather Data Load:
Previous rows: N/A
Current rows: 3182

Generation data frequency: 0 days 00:00:00
Weather data frequency: 0 days 00:15:00
Starting feature engineering...

Starting data resampling and alignment...

Generation data columns after aggregation:
['DATE_TIME', 'DC_POWER_sum', 'DC_POWER_mean', 'DC_POWER_mean_std', 'DC_POWER_mean_min', 'DC_POWER_mean_max', 'AC_POWER_sum', 'AC_POWER_mean', 'AC_POWER_mean_std', 'AC_POWER_mean_min', 'AC_POWER_mean_max', 'DC_POWER_std_mean', 'DC_POWER_min_min', 'DC_POWER_max_max', 'AC_POWER_std_mean', 'AC_POWER_min_min', 'AC_POWER_max_max', 'DAILY_YIELD', 'TOTAL_YIELD']

Generation Data Aggregation:
Previous rows: 68778
Current rows: 3158
Reduction: 65620 rows (95.41%)

Weather Data Interpolation:
Previous rows: 3182
Current rows: 3264
Reduction: -82 rows (-2.58%)

Final Merge and Cleaning:
Previous rows: 3158
Current rows: 3158
Reduction: 0 rows (0.00%)

Shape before

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 16, 128)             │          87,040 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 16, 128)             │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 16, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 16, 64)              │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 16, 64)              │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 16, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 32)                  │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 32)                  │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 32)                  │           1,056 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 32)                  │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 16)                  │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 151,489 (591.75 KB)

 Trainable params: 150,977 (589.75 KB)

 Non-trainable params: 512 (2.00 KB)

Epoch 1/50
34/35 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - loss: nan - mae: nan - mse: nan
Epoch 1: val_loss did not improve from inf
35/35 ━━━━━━━━━━━━━━━━━━━━ 13s 146ms/step - loss: nan - mae: nan - mse: nan - val_loss: nan - val_mae: nan - val_mse: nan - learning_rate: 5.0000e-04
Epoch 2/50
34/35 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - loss: nan - mae: nan - mse: nan
Epoch 2: val_loss did not improve from inf
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step - loss: nan - mae: nan - mse: nan - val_loss: nan - val_mae: nan - val_mse: nan - learning_rate: 5.0000e-04
Epoch 3/50
34/35 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: nan - mae: nan - mse: nan
Epoch 3: val_loss did not improve from inf
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 64ms/step - loss: nan - mae: nan - mse: nan - val_loss: nan - val_mae: nan - val_mse: nan - learning_rate: 5.0000e-04
Epoch 4/50
34/35 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: nan - mae: nan - mse: nan
Epoch 4: val_loss did not improve from inf
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 64ms/step -

Successfully saved rf model
Successfully saved xgb model
Successfully saved lstm model
Successfully saved plot: time_series_analysis
Successfully saved plot: correlation_analysis
Successfully saved plot: environmental_analysis
Successfully saved plot: distribution_analysis
Successfully saved plot: rolling_statistics
Successfully saved plot: prediction_comparison
Successfully saved plot: time_series
Successfully saved evaluation results

Execution complete. Results and plots have been saved.

Best Model: rf

Model Rankings:
rf: RMSE = 338.5488
ensemble: RMSE = 341.2745
xgb: RMSE = 443.6670
